In [1]:
import import_ipynb
from STGCN import Model
from tqdm import tqdm
import tensorflow as tf
import argparse
import inspect
import shutil
import yaml
import os
import pandas as pd

importing Jupyter notebook from STGCN.ipynb
importing Jupyter notebook from skeleton.ipynb


In [2]:
def get_parser():
    parser = argparse.ArgumentParser(
        description='Spatial Temporal Graph Convolutional Neural Network for Skeleton-Based Action Recognition')
    parser.add_argument(
        '--base-lr', type=float, default=1e-1, help='initial learning rate')
    parser.add_argument(
        '--num-classes', type=int, default=49, help='number of classes in dataset')
    parser.add_argument(
        '--batch-size', type=int, default=64, help='training batch size')
    parser.add_argument(
        '--num-epochs', type=int, default=30, help='total epochs to train')
    parser.add_argument(
        '--save-freq', type=int, default=10, help='periodicity of saving model weights')
    parser.add_argument(
        '--checkpoint-path',
        default="checkpoints",
        help='folder to store model weights')
    parser.add_argument(
        '--log-dir',
        default="logs",
        help='folder to store model-definition/training-logs/hyperparameters')
    parser.add_argument(
        '--train-data-path',
        default="tfrecord",
        help='path to folder with training dataset tfrecord files')
    parser.add_argument(
        '--test-data-path',
        default="tfrecord",
        help='path to folder with testing dataset tfrecord files')
    parser.add_argument(
        '--steps',
        type=int,
        default=[10, 40],
        nargs='+',
        help='the epoch where optimizer reduce the learning rate, eg: 10 50')
    parser.add_argument(
        '--gpus',
        default= None,
        nargs='+',
        help='list of gpus to use for training, eg: "/gpu:0" "/gpu:1"')

    return parser


In [3]:
def get_dataset(directory, num_classes=49, batch_size=32, drop_remainder=False,
                shuffle=False, shuffle_size=1000):
    # dictionary describing the features.
    feature_description = {
        'features': tf.io.FixedLenFeature([], tf.string),
        'label'     : tf.io.FixedLenFeature([], tf.int64)
    }

    # parse each proto and, the features within
    def _parse_feature_function(example_proto):
        features = tf.io.parse_single_example(example_proto, feature_description)
        data =  tf.io.parse_tensor(features['features'], tf.float32)
        index = tf.one_hot(features['label'], 2959)
        label = tf.one_hot(features['label'], num_classes)
        data = tf.reshape(data, (3, 16, 20, 1))
        return data, label, index

    records = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith("tfrecord")]
    dataset = tf.data.TFRecordDataset(records, num_parallel_reads=len(records))
    dataset = dataset.map(_parse_feature_function)
    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
    dataset = dataset.prefetch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(shuffle_size)
    return dataset

In [4]:
@tf.function
def train_step(features, labels):
    def step_fn(features, labels):
        with tf.GradientTape() as tape:
            logits = model(features, training=True)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                              labels=labels)
            loss = tf.reduce_sum(cross_entropy) * (1.0 / global_batch_size)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        train_acc(labels, logits)
        train_acc_top_5(labels, logits)
        cross_entropy_loss(loss)
    strategy.run(step_fn, args=(features, labels,))

In [5]:
@tf.function
def test_step(features):
    logits = model(features, training=False)
    return tf.nn.softmax(logits)

In [6]:
if __name__ == "__main__":
    parser = get_parser()
    arg, unknown = parser.parse_known_args()

    base_lr         = arg.base_lr
    num_classes     = arg.num_classes
    epochs          = arg.num_epochs
    checkpoint_path = arg.checkpoint_path
    log_dir         = arg.log_dir
    train_data_path = arg.train_data_path
    test_data_path  = arg.test_data_path
    save_freq       = arg.save_freq
    steps           = arg.steps
    batch_size      = arg.batch_size
    gpus            = arg.gpus
    strategy        = tf.distribute.MirroredStrategy(arg.gpus)
    global_batch_size = arg.batch_size*strategy.num_replicas_in_sync
    arg.gpus        = strategy.num_replicas_in_sync

    #copy hyperparameters and model definition to log folder
    #save_arg(arg)
    shutil.copy2(inspect.getfile(Model), arg.log_dir)
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    '''
    Get tf.dataset objects for training and testing data
    Data shape: features - batch_size, 3, 300, 25, 2
                labels   - batch_size, num_classes
    '''
    train_data = get_dataset(train_data_path,
                             num_classes=num_classes,
                             batch_size=global_batch_size,
                             drop_remainder=True,
                             shuffle=True)
    
    train_data = strategy.experimental_distribute_dataset(train_data)

    test_data = get_dataset(test_data_path,
                            num_classes=num_classes,
                            batch_size=batch_size,
                            drop_remainder=False,
                            shuffle=False)

    boundaries = [(step*40000)//batch_size for step in steps]
    values = [base_lr]*(len(steps)+1)
    for i in range(1, len(steps)+1):
        values[i] *= 0.1**i
    learning_rate  = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

    with strategy.scope():
        model        = Model(num_classes=num_classes)
        optimizer    = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                               momentum=0.9,
                                               nesterov=True)
        ckpt         = tf.train.Checkpoint(model=model, optimizer=optimizer)
        ckpt_manager = tf.train.CheckpointManager(ckpt,
                                                  checkpoint_path,
                                                  max_to_keep=5)

        # keras metrics to hold accuracies and loss
        cross_entropy_loss   = tf.keras.metrics.Mean(name='cross_entropy_loss')
        train_acc            = tf.keras.metrics.CategoricalAccuracy(name='train_acc')
        train_acc_top_5      = tf.keras.metrics.TopKCategoricalAccuracy(name='train_acc_top_5')

    epoch_test_acc       = tf.keras.metrics.CategoricalAccuracy(name='epoch_test_acc')
    epoch_test_acc_top_5 = tf.keras.metrics.TopKCategoricalAccuracy(name='epoch_test_acc_top_5')
    test_acc_top_5       = tf.keras.metrics.TopKCategoricalAccuracy(name='test_acc_top_5')
    test_acc             = tf.keras.metrics.CategoricalAccuracy(name='test_acc')
    summary_writer       = tf.summary.create_file_writer(log_dir)

    # Get 1 batch from train dataset to get graph trace of train and test functions
    
    for data in test_data:
        features, labels, index = data
        break

    # add graph of train and test functions to tensorboard graphs
    # Note:
    # graph training is True on purpose, allows tensorflow to get all the
    # variables, which is required for the first call of @tf.function function
    tf.summary.trace_on(graph=True)
    train_step(features, labels)
    with summary_writer.as_default():
        tf.summary.trace_export(name="training_trace",step=0)
    tf.summary.trace_off()

    tf.summary.trace_on(graph=True)
    test_step(features)
    with summary_writer.as_default():
        tf.summary.trace_export(name="testing_trace", step=0)
    tf.summary.trace_off()

    # start training
    train_iter = 0
    test_iter = 0
    for epoch in range(epochs):
        print("Epoch: {}".format(epoch+1))
        print("Training: ")
        with strategy.scope():
            for features, labels, index in tqdm(train_data):
                train_step(features, labels)
                with summary_writer.as_default():
                    tf.summary.scalar("cross_entropy_loss",
                                      cross_entropy_loss.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc",
                                      train_acc.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc_top_5",
                                      train_acc_top_5.result(),
                                      step=train_iter)
                cross_entropy_loss.reset_states()
                train_acc.reset_states()
                train_acc_top_5.reset_states()
                train_iter += 1

        print("Testing: ")
        for features, labels, index in tqdm(test_data):
            y_pred = test_step(features)
            print("test acc ", test_acc(labels, y_pred))
            print("epoch test acc ", epoch_test_acc(labels, y_pred))
            print("test acc top5", test_acc_top_5(labels, y_pred))
            print("epoch test acc top5 ",epoch_test_acc_top_5(labels, y_pred))
            with summary_writer.as_default():
                tf.summary.scalar("test_acc",
                                  test_acc.result(),
                                  step=test_iter)
                tf.summary.scalar("test_acc_top_5",
                                  test_acc_top_5.result(),
                                  step=test_iter)
            test_acc.reset_states()
            test_acc_top_5.reset_states()
            test_iter += 1
        with summary_writer.as_default():
            tf.summary.scalar("epoch_test_acc",
                              epoch_test_acc.result(),
                              step=epoch)
            tf.summary.scalar("epoch_test_acc_top_5",
                              epoch_test_acc_top_5.result(),
                              step=epoch)
        epoch_test_acc.reset_states()
        epoch_test_acc_top_5.reset_states()

        if (epoch + 1) % save_freq == 0:
            ckpt_save_path = ckpt_manager.save()
            print('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                ckpt_save_path))

    ckpt_save_path = ckpt_manager.save()
    print('Saving final checkpoint for epoch {} at {}'.format(epochs,

                                                              ckpt_save_path))
    
    test_data = get_dataset("tfrecord_test",
                            num_classes=49,
                            batch_size=64,
                            drop_remainder=False,
                            shuffle=False)
    result=[]
    index_list=[]
    for features, labels, index in tqdm(test_data):
        pred_index = tf.keras.backend.eval(tf.math.argmax(index, 1))
        y_pred = test_step(features)
        pred_label = tf.keras.backend.eval(tf.math.argmax(y_pred, 1))
        for item in pred_label:
            result.append(item)
        for item in pred_index:
            index_list.append(item)

    
    print("----------------------------------------------")
    Id = [i for i in range (0, 2959)]
    predict = pd.DataFrame(Id, columns = ['Id'])
    result_order = [None]  * 2959
    for i, item in enumerate(index_list):
        result_order[item] = result[i]
    print(result_order)
    predict['Category'] = result_order
    print(predict)
    predict.to_csv('output.csv', index=False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Num GPUs Available:  1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:loca

271it [00:37,  7.22it/s]


Testing: 


2it [00:00, 12.65it/s]

test acc  tf.Tensor(0.09375, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.09375, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.421875, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.421875, shape=(), dtype=float32)
test acc  tf.Tensor(0.0625, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.078125, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.34375, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.3828125, shape=(), dtype=float32)
test acc  tf.Tensor(0.078125, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.078125, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.328125, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.36458334, shape=(), dtype=float32)
test acc  

...

272it [00:15, 18.05it/s]

test acc  tf.Tensor(0.8125, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.49407527, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.96875, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.64242566, shape=(), dtype=float32)
test acc  tf.Tensor(0.875, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.4954861, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.984375, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.64369214, shape=(), dtype=float32)
test acc  tf.Tensor(0.796875, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.49659824, shape=(), dtype=float32)
test acc top5 tf.Tensor(1.0, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.6450069, shape=(), dtype=float32)
test acc  tf.Tensor(0.71428573, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.49668607, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.85714287, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.6450925, shape=(), dtype=float32)


Saving checkpoint for epoch 30 at checkpoints\ckpt-3
Saving final checkpoint for epoch 30 at checkpoints\ckpt-4


47it [00:02, 16.83it/s]

----------------------------------------------
[1, 11, 9, 3, 28, 2, 17, 9, 31, 3, 27, 26, 46, 25, 3, 34, 34, 8, 22, 6, 27, 7, 27, 16, 21, 14, 3, 4, 20, 2, 43, 8, 1, 20, 5, 2, 6, 25, 3, 8, 11, 19, 3, 18, 23, 22, 10, 17, 12, 17, 7, 8, 18, 7, 9, 14, 8, 10, 12, 15, 34, 1, 4, 26, 8, 6, 5, 17, 21, 1, 36, 13, 9, 5, 12, 3, 10, 1, 22, 1, 21, 7, 30, 27, 8, 1, 40, 8, 24, 10, 9, 18, 8, 43, 23, 7, 17, 8, 15, 5, 30, 19, 3, 40, 29, 6, 10, 19, 7, 32, 40, 20, 8, 18, 31, 4, 13, 9, 21, 10, 8, 14, 27, 3, 12, 8, 42, 3, 13, 10, 17, 44, 41, 14, 1, 19, 12, 7, 3, 20, 8, 13, 30, 2, 12, 29, 30, 22, 6, 20, 17, 25, 36, 39, 1, 3, 4, 46, 7, 1, 15, 6, 29, 10, 3, 18, 18, 6, 8, 45, 28, 14, 13, 12, 9, 17, 6, 3, 13, 10, 32, 43, 19, 8, 37, 46, 26, 25, 9, 9, 8, 4, 11, 10, 6, 9, 21, 9, 27, 6, 13, 23, 14, 8, 7, 8, 10, 26, 2, 36, 30, 3, 46, 8, 3, 3, 13, 8, 7, 19, 25, 9, 40, 28, 11, 1, 2, 7, 8, 9, 6, 1, 3, 40, 46, 8, 37, 25, 18, 35, 14, 6, 26, 4, 9, 18, 14, 29, 30, 13, 11, 10, 2, 29, 8, 27, 23, 6, 10, 39, 8, 1, 6, 8, 10, 13, 1